### Configure the Model

In [6]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")
output = model.invoke("Hi, how are you")
print(output)

content="I'm just a language model, I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 15, 'total_tokens': 61, 'completion_time': 0.083256357, 'prompt_time': 0.003768767, 'queue_time': 0.128422542, 'total_time': 0.087025124}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8b7c3a83f7', 'finish_reason': 'stop', 'logprobs': None} id='run--89544bc3-a6b5-4feb-bf1b-f246941c813b-0' usage_metadata={'input_tokens': 15, 'output_tokens': 46, 'total_tokens': 61}


### Configure the Embeddings

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("HI"))

c:\Users\Anilchoudary R\Gitrepos\langgraph_playground\langgraph_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


384

### Lets take a data embedded and store it in the Vector DB

In [18]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [25]:
## Data Loader
loader=DirectoryLoader("../data",glob="./*.txt", loader_cls=TextLoader)

In [29]:
docs = loader.load()
docs

[Document(metadata={'source': '..\\data\\indian_economy.txt'}, page_content="The economy of India is a developing mixed economy with a notable public sector in strategic sectors.[52] It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP).[53] From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj.[54][55] The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning.[56][57] India has about 1,900 public sector companies,[58] with the Indian state having complet

In [31]:
docs[0].page_content

"The economy of India is a developing mixed economy with a notable public sector in strategic sectors.[52] It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP).[53] From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj.[54][55] The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning.[56][57] India has about 1,900 public sector companies,[58] with the Indian state having complete control and ownership of railways and highways. The Indian government has 

In [34]:
## Chunking data 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)


In [37]:
new_docs =text_splitter.split_documents(documents=docs)

In [40]:
new_docs

[Document(metadata={'source': '..\\data\\indian_economy.txt'}, page_content="The economy of India is a developing mixed economy with a notable public sector in strategic sectors.[52] It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing"),
 Document(metadata={'source': '..\\data\\indian_economy.txt'}, page_content='nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP).[53] From independence in 1947 until 1991,'),
 Document(metadata={'source': '..\\data\\indian_economy.txt'}, page_content='(PPP).[53] From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention,'),
 Document(metadata={'source': '..\\data\\indian_economy.txt'}, page_content='with extensive Sovietization, state intervention, demand-side economics, natural resources,

In [43]:
doc_string = [doc.page_content for doc in new_docs]
doc_string

["The economy of India is a developing mixed economy with a notable public sector in strategic sectors.[52] It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing",
 'nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP).[53] From independence in 1947 until 1991,',
 '(PPP).[53] From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention,',
 'with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of',
 'This is characterised as dirigism, in the form of the Licence Raj.[54][55] The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic',
 'in

In [44]:
len(doc_string)

27